# scVI

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 05.01.2023
- **Date of Last Modification:** 26.07.2024 (Sebastian Birk; <sebastian.birk@helmholtz-munich.de>)

- The scVI source code is available at https://github.com/scverse/scvi-tools.
- The corresponding publication is "Lopez, R., Regier, J., Cole, M. B., Jordan, M. I. & Yosef, N. Deep generative modeling for single-cell transcriptomics. Nat. Methods 15, 1053–1058 (2018)".
- The workflow of this notebook follows the tutorial from https://docs.scvi-tools.org/en/stable/tutorials/notebooks/api_overview.html.

Run this notebook in the nichecompass-reproducibility environment, installable from ```('../../../envs/environment.yaml')```. 

## 1. Setup

### 1.1 Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import scipy.sparse as sp
import scvi
import squidpy as sq

ModuleNotFoundError: No module named 'jax.linear_util'

### 1.2 Define Parameters

In [ ]:
model_name = "scvi"
latent_key = f"{model_name}_latent"

### 1.3 Run Notebook Setup

In [ ]:
sc.set_figure_params(figsize=(6, 6))

In [ ]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [ ]:
data_folder_path = "../../../datasets/st_data/gold/"
benchmarking_folder_path = "../../../artifacts/single_sample_method_benchmarking"
figure_folder_path = f"../../../figures"

## 2. scVI Model

### 2.1 Define Training Function

In [ ]:
def train_scvi_models(dataset,
                      cell_type_key,
                      niche_type_key=None,
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16]):
    # Configure figure folder path
    dataset_figure_folder_path = f"{figure_folder_path}/{dataset}/single_sample_method_benchmarking/" \
                                 f"{model_name}/{current_timestamp}"
    os.makedirs(dataset_figure_folder_path, exist_ok=True)
    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:
        adata_original = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        if niche_type_key in adata_original.obs.columns:
            adata_new.obs["niche_type"] = adata_original.obs[niche_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        del(adata_original)
    
    model_seeds = list(range(10))
    for run_number, n_neighbors in zip(np.arange(n_start_run, n_end_run+1), n_neighbor_list):
        # n_neighbors is here only used for the latent neighbor graph construction used for
        # UMAP generation and clustering as scVI is not a spatial method
        
        # Load data
        adata = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
        
        start_time = time.time()
        
        scvi.settings.seed = model_seeds[run_number-1]

        # Setup adata
        scvi.model.SCVI.setup_anndata(adata,
                                      layer="counts")

        # Initialize model
        model = scvi.model.SCVI(adata)

        # Train model
        model.train()

        adata.obsm[latent_key] = model.get_latent_representation()
        
        # Measure time for model training
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Duration of model training in run {run_number}: "
              f"{int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds.")
        adata_new.uns[f"{model_name}_model_training_duration_run{run_number}"] = (
            elapsed_time)
        
        # Store latent representation
        adata_new.obsm[latent_key + f"_run{run_number}"] = adata.obsm[latent_key]

        # Store intermediate adata to disk
        adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad")  

    # Store final adata to disk
    adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad") 

### 2.2 Train Models on Benchmarking Datasets

In [ ]:
train_scvi_models(dataset="seqfish_mouse_organogenesis_embryo2",
                  cell_type_key="celltype_mapped_refined",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"seqfish_mouse_organogenesis_subsample_{subsample_pct}pct_embryo2",
                      cell_type_key="celltype_mapped_refined",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="nanostring_cosmx_human_nsclc_batch5",
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"nanostring_cosmx_human_nsclc_subsample_{subsample_pct}pct_batch5",
                      cell_type_key="cell_type",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="vizgen_merfish_mouse_liver",
                  cell_type_key="Cell_Type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"vizgen_merfish_mouse_liver_subsample_{subsample_pct}pct",
                      cell_type_key="Cell_Type",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="slideseqv2_mouse_hippocampus",
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"slideseqv2_mouse_hippocampus_subsample_{subsample_pct}pct",
                      cell_type_key="cell_type",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="sim1_1105genes_10000locs_strongincrements",
                  cell_type_key="cell_types",
                  niche_type_key="niche_types",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="starmap_mouse_mpfc",
                  cell_type_key="cell_type",
                  niche_type_key="niche_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
train_scvi_models(dataset="stereoseq_mouse_embryo",
                  cell_type_key="leiden",
                  niche_type_key="niche_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])